In [2]:
%cd ..

/Users/nguyenhuyhai/20221/lab_blockchain/orchai_validator


In [3]:
from labeling.upload import ETLProcessor, get_spark, get_batch_intervals, sampling
from omegaconf import OmegaConf

In [4]:
config = OmegaConf.load("config/local_full.yaml")

In [5]:
spark = get_spark()

23/01/19 13:20:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
def get_batch_intervals(start_block, end_block, batch_size, vote_proposed_win_size, label_win_size, min_block):
    intervals = []
    while start_block <= end_block:
        batch_start = max(min_block, start_block - vote_proposed_win_size + 1)
        batch_end = batch_start + batch_size - 1
        
        if batch_end > end_block:
            batch_size = end_block - batch_start + 1
            batch_size = (batch_size - vote_proposed_win_size + 1) // combine_win_size * combine_win_size + vote_proposed_win_size - 1
            batch_end = batch_start + batch_size - 1
            
            if batch_start > batch_end:
                break
            
            if batch_size < vote_proposed_win_size + combine_win_size + label_win_size - 1:
                break

        intervals.append((batch_start, batch_end))
        start_block = batch_end - label_win_size + 1
    
    return intervals


vote_proposed_win_size = config.hp.etl.vote_proposed_win_size
label_win_size = config.hp.etl.label_win_size
batch_size = config.hp.upload.batch_size

for bs, be in get_batch_intervals(
    start_block=7063372,
    end_block=7065370,
    batch_size=batch_size,
    vote_proposed_win_size=vote_proposed_win_size,
    label_win_size=label_win_size,
    min_block=7049472
):
    print(bs, be)

KeyboardInterrupt: 

In [ ]:
7064871 - 7062873 + 1

In [6]:
df = sampling(spark, config, 7062873, 7064871)

Successfully queried data from database


In [ ]:
df.groupBy("block_height").count().count()

In [7]:
dff = ETLProcessor.data_scoring(df, **config.hp.etl)

------------------------------------------------
Successfully converted voting_power_score column
------------------------------------------------
Successfully converted commission_score column
------------------------------------------------
Successfully converted self_bonded_score column
------------------------------------------------
Successfully converted vote_propose_score column
------------------------------------------------
Sucessfully converted final_score
------------------------------------------------
Sucessfully shifting data
------------------------------------------------


In [8]:
dff.printSchema()

root
 |-- block_height: integer (nullable = true)
 |-- operator_address: string (nullable = true)
 |-- jailed: integer (nullable = true)
 |-- status: string (nullable = true)
 |-- tokens: decimal(38,0) (nullable = true)
 |-- commission_rate: double (nullable = true)
 |-- self_bonded: decimal(38,6) (nullable = true)
 |-- delegators_token: decimal(38,6) (nullable = true)
 |-- validators_count_per_block: long (nullable = true)
 |-- total_token_amt_per_block: decimal(38,0) (nullable = true)
 |-- total_self_bonded_amt_per_block: decimal(38,6) (nullable = true)
 |-- voting_power_score: decimal(38,6) (nullable = true)
 |-- commission_score: double (nullable = true)
 |-- self_bonded_score: double (nullable = true)
 |-- vote_score: double (nullable = true)
 |-- score: double (nullable = true)
 |-- label: double (nullable = true)



23/01/19 17:30:05 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 578620 ms exceeds timeout 120000 ms
23/01/19 17:30:05 WARN SparkContext: Killing executors is not supported by current scheduler.


In [ ]:
dff.groupBy("block_height").count().count()

In [ ]:
dp = dff.groupBy("block_height").count().pandas_api()

In [ ]:
d = dp.values

In [ ]:
bh = d[:, 0]

In [ ]:
bh.sort()

In [ ]:
import numpy as np
from collections import Counter

cnt = Counter(np.diff(bh))
cnt

In [ ]:
bh[0], bh[-1]

In [ ]:
7063372, 7064792
7062873, 7064871

In [ ]:
7063372 - 7062873 + 1

In [ ]:
7064871 - 7064792 + 1